In [1]:
! pip install -Uqqq pip --progress-bar off
! pip install -qqq langchain==0.0.228 --progress-bar off
! pip install -qqq chromadb==0.3.26 --progress-bar off
! pip install -qqq sentence-transformers==2.2.2 --progress-bar off
! pip install -qqq auto-gptq==0.2.2 --progress-bar off
! pip install -qqq einops==0.6.1 --progress-bar off
! pip install -qqq unstructured==0.8.1 --progress-bar off
! pip install -qqq transformers==4.30.2 --progress-bar off
! pip install -qqq torch==2.0.1 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.31.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [2]:
from pathlib import Path
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, pipeline , TextStreamer

In [3]:
questions_dir=Path("HRPolicies")
questions_dir.mkdir(exist_ok=True,parents=True)

def write_file(question, answer, file_path):
  text=f"""
  "Q":{question}
  "A":{answer}""".strip()
  with Path(questions_dir / file_path).open("w") as text_file:
    text_file.write(text)


In [4]:
write_file(question="what is your salary or cost to the company",
          answer='Total Salary (Cost To Company) : 27,50,000'.strip(),
           file_path="text1.txt",)

In [5]:
write_file(question="what is your notice period",
          answer="""Termination/Separation
9. The employment can be terminated by either of the parties by giving 2 months notice period in writing of its intention to do so. An employee is expected to serve the full duration of the notice period to facilitate proper hand over of responsibilities and complete any urgent projects on hand. In exceptional cases, based on Company's discretion, an employee may be permitted to be relieved earlier by tendering notice pay calculated as basic pay divided by 30 and multiplied by number of days of shortfall in notice period. The basic salary is defined as the first element of our existing salary structure
10. In the event of breach of any of the terms & conditions of your employment and/or service conditions and rules, the Company reserves the right to claim damages from you. The Company also reserves the right to terminate your services without giving notice.
11. Your appointment and employment will be subject to your being and remaining medically fit. It is necessary for you to get medically examined by the Medical Officer appointed by the Company as and when required by the Company. If at any stage, you are found to be unfit by the Medical Officer for the job currently being done by you, then you are liable to be terminated on medical grounds.
12. You will retire from the employment of this Company upon reaching the age of 60 years.""".strip(),
           file_path="text2.txt",)

In [6]:
write_file(question="what is your retirement age from the company",
          answer='You will retire from the employment of this Company upon reaching the age of 60 years'.strip(),
           file_path="text3.txt",)

In [7]:
write_file(question="what is your responsibilities?",
          answer="""B) Responsibility
3. During your employment with the Company, you will be governed by Service Conditions more specifically stated in the Annexure
I attached to this letter and other rules and regulations framed by the Management from time to time, which shall be applicable to
you, and also by such legal provisions as may be applicable. You shall perform the roles and responsibility as mentioned in the
Annexure III attached to this letter.
4. You are expected to give to the Company your best efforts, attention and commitment. You are explicitly advised to refrain from
any such activity, whether for monetary or any other considerations, as may become, in our opinion, a hindrance to your
performance . You are, by virtue of employment with this Company, required to do all the work allied, ancillary related or incidental to the
main job. Similarly, you may be asked to do any job within your competence depending upon the exigencies of the situation.
6. Your appointment is a full time job and you will not at any time engage, directly or indirectly, in any paid or honorary
occupation or business outside the Company without obtaining prior written consent of the Company. You shall not by
yourself or on behalf of the Company conduct any external training/s workshops without prior approval from the Company.
Breach of any of the above conditions will render your employment liable to termination without notice.""".strip(),
           file_path="text4.txt",)

In [8]:
write_file(question="does company provide any training?",
          answer="""Training
7. In furtherance of your employment in this Company and for increasing and honing your skills, you may be required to be
trained technically or otherwise. This may require the Company to provide training either in-house or send you for training
outside the Company in India or abroad, on the terms and conditions as mutually agreed before the commencement of such
training.
""".strip(),
           file_path="text5.txt",)

In [9]:
write_file( question="under what circumstances you can terminated? ",
          answer=""" If at any
stage, you are found to be unfit by the Medical Officer for the job currently being done by you, then you are liable to be
terminated on medical grounds. You will automatically retire from the employment of this Company upon reaching the age of
60 years. 11. Your employment is substantially based on the information, testimonial, documents submitted by you, which you were
required to submit while joining the employment or may be called upon to do so subsequently. The Company shall be within
its right to verify the correctness of such information at any time now or in future. If it is found at any stage that the
information provided by you is incorrect or in case some information is suppressed, then the Company reserves the right to
terminate your services without giving any notice. 12. Your appointment is subject to favourable background screening check report. That the management shall terminate your
services forthwith at its discretion in case there is any adverse report while checking the background or in case any
information is concealed or is found to be false or misleading. So also your services are liable to be terminated without notice
in case while in employment you are involved in any act of moral turpitude 13. You will be solely responsible for the Company property assigned to you to discharge your duties. Loss of any of the items
would be recovered from you, as the Company may deem appropriate. On ceasing to be in the employment of this Company
for any reason, you will promptly settle all the accounts including the return of all Company properties, tools, equipment,
documents, etc. without making or retaining any copies """.strip(),
           file_path="text6.txt",)

In [10]:
write_file(question="what is company notice period?",
          answer="""Termination Separation 8. The employment can be terminated by either of the parties by giving 2 month’s notice in writing informing the other party of
its intention to do so.
""".strip(),
           file_path="text10.txt",)

In [11]:
write_file(question="who much pay you get if you relieved earlier?",
          answer="""ased on Company's discretion, you may
be permitted to be relieved earlier by tendering notice pay calculated as basic pay /30 * no. of days of shortfall in notice
period. The basic salary is defined as the first element of our existing salary structure 9. In the event of breach of any of the terms & conditions of your employment and/or service conditions and rules, the
Company reserves the right to claim damages from you. The Company also reserves the right to terminate your services without giving notice.
10. Your appointment and employment will be subject to your being and remaining medically fit
""".strip(),
           file_path="text11.txt",)

In [12]:
write_file(question="Do I need to inform the company if I changes the residential address",
          answer="""You shall always endeavor to upgrade your skills, knowledge, and expertise from time to time and shall not refuse to undergo
any training or programme undertaken by Company or as directed by the Company for improvement or upgradation of skills,
services performance or such other things necessary for the growth of the Company.
11. Upon leaving the employment you shall return to the Company forthwith all the property, documents drawings, designs,
programmes, data in whatever form, hardware, software, records, etc belonging to the Company or its associates, subsidiary,
clients, or customers.
12. If at any time, whether during or after your association with us, any dispute arises between you and/or your legal heirs and
representative on the one hand and ourselves and/or our assignees and/or successors on the other, whether regarding
interpretation and/or legal effect of all or any of the terms of this agreement and/or whether or not any breach of it was or is
committed by either of us and whether or not the said agreement or any of its terms are reasonable and/or regarding nature,
type, extent and/or quantum of relief, all and every one of them shall be settled by Conciliation and Binding Arbitration
at Pune/Delhi under the Indian Arbitration & conciliation act 1996 by the sole arbitrator appointed by the company having
jurisdiction at Pune, and further that you shall not be entitled to take any of the disputes to any other place and/or to any Civil
Court.
13. All the correspondence, communications by the Company hereinafter shall be made either personally at work place or at the
residential address given by you, at any one of the places at the discretion and convenience of the Company. Should you
change your residence, you shall forthwith inform the address in writing to the Company.
""".strip(),
           file_path="text8.txt",)

In [13]:
write_file(question="what is intellectual property related clauses",
          answer="""AGREEMENT CONCERNING EMPLOYEE INVENTIONS. INTELLECTUAL PROPERTY
PERSONAL DATA AND CONFIDENTIAL INFORMATION
In consideration of my employment, the continuation of my employment and/or other consideration I have received or will
receive in connection with my employment by XYZ Technologies Private Limited or any of its affiliates, including
subsidiaries, partnerships or entities heretofore or hereafter controlling, controlled by, or under common control with it
(collectively "XYZ") """.strip(),
           file_path="text12.txt",)



In [14]:
DEVICE="cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda:0


In [15]:
model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
model_basename = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

In [16]:
generation_config=GenerationConfig.from_pretrained(model_name_or_path)

In [17]:
question=(" which language is better to understand for the beginner python or java?")

prompt=f"""
### instruction:{question}
### response: """.strip()

print(prompt)

### instruction: which language is better to understand for the beginner python or java?
### response:


In [18]:
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda(DEVICE)
with torch.inference_mode():
  output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

In [19]:
print(tokenizer.decode(output[0]))

<s> ### instruction: which language is better to understand for the beginner python or java?
### response:It is subjective to say which language is better for a beginner as it depends on the individual's learning style, preferences, and goals. Both Python and Java are popular programming languages with their own strengths and weaknesses. Python is known for its readability and simplicity, making it a great choice for beginners. Java, on the other hand, is known for its versatility and scalability, making it a popular choice for enterprise-level applications. Ultimately, the choice depends on the individual's goals and interests.</s>


In [20]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.30.2"
}

In [21]:
streamer=TextStreamer(tokenizer,skip_prompt=True, skip_special_tokens=True,use_multiprocessing=False)

In [22]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1,
)

print(pipe(prompt)[0]['generated_text'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Peg

It depends on personal preference and career goals. Python is generally easier to learn, but Java has a wider range of applications in the industry.
### instruction: which language is better to understand for the beginner python or java?
### response:It depends on personal preference and career goals. Python is generally easier to learn, but Java has a wider range of applications in the industry.


In [23]:
llm=HuggingFacePipeline(pipeline=pipe)

In [24]:
response=llm(prompt)

It depends on personal preference and career goals. Python is generally easier to learn, but Java has a wider range of applications in the industry.


In [25]:
embeddings= HuggingFaceEmbeddings(
    model_name="embaas/sentence-transformers-multilingual-e5-base",
    model_kwargs= {"device": DEVICE},
)

In [26]:
loader=DirectoryLoader("./HRPolicies/",glob="**/*txt")
documets=loader.load()
len(documets)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


10

In [27]:
textsplitter=CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
texts=textsplitter.split_documents(documets)

In [28]:
texts[0]

Document(page_content='"Q":what is your retirement age from the company "A":You will retire from the employment of this Company upon reaching the age of 60 years', metadata={'source': 'HRPolicies/text3.txt'})

In [29]:
db=Chroma.from_documents(texts,embeddings)

In [30]:
db.similarity_search("notice period")

[Document(page_content='"Q":what is company notice period? "A":Termination Separation 8. The employment can be terminated by either of the parties by giving 2 month’s notice in writing informing the other party of its intention to do so.', metadata={'source': 'HRPolicies/text10.txt'}),
 Document(page_content='"Q":what is your notice period "A":Termination/Separation 9. The employment can be terminated by either of the parties by giving 2 months notice period in writing of its intention to do so. An employee is expected to serve the full duration of the notice period to facilitate proper hand over of responsibilities and complete any urgent projects on hand. In exceptional cases, based on Company\'s discretion, an employee may be permitted to be relieved earlier by tendering notice pay calculated as basic pay divided by 30 and multiplied by number of days of shortfall in notice period. The basic salary is defined as the first element of our existing salary structure 10. In the event of 

In [31]:
template="""
### Instruction: your are customer support agent  talking to employees for answer Human Rerource policies related queries. use
only chat history and following  information {context}
to anser helpful manner to the question. if you dont know the answer say that - you dont know.
keep your reply short and informative.
{chat_history}
### Input : {question}
### Response: """.strip()


In [33]:
prompt=PromptTemplate(input_variables=['question','context','chat_history'], template=template)

In [34]:
memory=ConversationBufferMemory(
      memory_key='chat_history',
      human_prefix="### Input",
      ai_prefix='### Response',
      output_key='answer',
      return_messages=True
)

In [36]:
chain=ConversationalRetrievalChain.from_llm(llm,
                                            chain_type="stuff",
                                            retriever=db.as_retriever(),
                                            memory=memory,
                                            combine_docs_chain_kwargs={'prompt':prompt},
                                            return_source_documents=True,)

In [37]:
question=" what is notice period"
answer=chain(question)

The notice period refers to the amount of time an employee must provide before leaving their job. This typically ranges from two weeks to several months, depending on the company's policy and the employee's position.


In [38]:
answer.keys()

dict_keys(['question', 'chat_history', 'answer', 'source_documents'])

In [40]:
answer['source_documents']

[Document(page_content='"Q":what is company notice period? "A":Termination Separation 8. The employment can be terminated by either of the parties by giving 2 month’s notice in writing informing the other party of its intention to do so.', metadata={'source': 'HRPolicies/text10.txt'}),
 Document(page_content='"Q":what is your notice period "A":Termination/Separation 9. The employment can be terminated by either of the parties by giving 2 months notice period in writing of its intention to do so. An employee is expected to serve the full duration of the notice period to facilitate proper hand over of responsibilities and complete any urgent projects on hand. In exceptional cases, based on Company\'s discretion, an employee may be permitted to be relieved earlier by tendering notice pay calculated as basic pay divided by 30 and multiplied by number of days of shortfall in notice period. The basic salary is defined as the first element of our existing salary structure 10. In the event of 

In [49]:
#QA chain with memeory

memory=ConversationBufferMemory(
      memory_key='chat_history',
      human_prefix="### Input",
      ai_prefix='### Response',
      input_key='question',
      output_key='output_text',
      return_messages=True,
)

chain=load_qa_chain(
    llm,
    chain_type="stuff",
    prompt=prompt,
    memory=memory,
    verbose=True
)

In [51]:
question=' what is company notice period?'
docs=db.similarity_search(question)
answer=chain.run({"input_documents":docs,"question": question})




> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
### Instruction: your are customer support agent  talking to employees for answer Human Rerource policies related queries. use
only chat history and following  information "Q":what is company notice period? "A":Termination Separation 8. The employment can be terminated by either of the parties by giving 2 month’s notice in writing informing the other party of its intention to do so.

"Q":what is your notice period "A":Termination/Separation 9. The employment can be terminated by either of the parties by giving 2 months notice period in writing of its intention to do so. An employee is expected to serve the full duration of the notice period to facilitate proper hand over of responsibilities and complete any urgent projects on hand. In exceptional cases, based on Company's discretion, an employee may be permitted to be relieved earlier by tendering notice pay calculated as basic pay divided by 30 and multipli

In [52]:
question=' does company provide trainings?'
docs=db.similarity_search(question)
answer=chain.run({"input_documents":docs,"question": question})



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
### Instruction: your are customer support agent  talking to employees for answer Human Rerource policies related queries. use
only chat history and following  information "Q":does company provide any training? "A":Training 7. In furtherance of your employment in this Company and for increasing and honing your skills, you may be required to be trained technically or otherwise. This may require the Company to provide training either in-house or send you for training outside the Company in India or abroad, on the terms and conditions as mutually agreed before the commencement of such training.

"Q":what is your responsibilities? "A":B) Responsibility 3. During your employment with the Company, you will be governed by Service Conditions more specifically stated in the Annexure I attached to this letter and other rules and regulations framed by the Management from time to time, which shall be applicable to you, 